# Что нейронные сети знают о наших лицах?

![](screens/mKWoBR5P5t4.jpg)

### [Школа GoTo](https://goto.msk.ru)
[Емельяненко Дмитрия](https://github.com/TIXFeniks) <br>
[Творожков Андрей](https://tvorog.me)


### Подключим необходимые библиотеки

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from helpers.lfw_dataset import load_lfw_dataset
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import LSHForest
from IPython.display import display
from ipywidgets import widgets
from helpers.autoencoder import load_autoencoder
import skimage
from skimage import io
import matplotlib.patches as patches
from skimage import transform


# Загрузим датасет
Данные были уже загружены специально для вас. Ссылки (на всякий случай):
- http://www.cs.columbia.edu/CAVE/databases/pubfig/download/lfw_attributes.txt
- http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
- http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [ ]:
## Загружаем датасет
X, attr = load_lfw_dataset(use_raw=True,dimx=38,dimy=38)

# Представляем ввиде матрицы
m_attr = attr.as_matrix()

# Делаем непонятно что
X = X.astype('float32') / 255.0

# Смотрим размерность картинки
img_shape = X.shape[1:]

# Делим на трейн, тест
X_train, X_test, attr_train, attr_test = train_test_split(X, m_attr, test_size=0.1, random_state=42)

## Посмотрим на имеющиеся изображения

In [ ]:
plt.title('sample image')
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(X[i])

print("X shape:",X.shape)
print("attr shape:",attr.shape)

##  Модель

Для манипуляций с лицами мы будем использовать автокодировщик. Эту модель мы учим сжимать картинку до вектора малой размерности и разжимать её обратно, теряя как можно меньше инфромации.

<img src="https://blog.keras.io/img/ae/autoencoder_schema.jpg">

In [ ]:
autoencoder = load_autoencoder(img_shape, weights_file= 'model_weights/deep_weights_64.pkl')
# gan4.pkl - это весело! Попробуй и другие .pkl файлы из папки с этой тетрадкой

За время мастер-класса мы не успеем обучить модель с нуля, так что мы предобучили модель заранее, код на предыдущей клетке загружает модель.

![](screens/dl_meme2.jpg)

## Визуализация

Используем нашу модель для того, чтобы сжать картинки. Затем разожмём их обратно

In [ ]:
def visualize(img,autoencoder):
    """Draws original, encoded and decoded images"""
    code = autoencoder.encode(img[None])[0]
    reco = autoencoder.decode(code[None])[0]
    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img)

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//8,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco.clip(0,1))
    plt.show()


In [ ]:
for i in range(10):
    img = X_test[i]
    visualize(img,autoencoder)


## Поиск картинок с помощью автокодировщиков
Нам удалось научить модель сжимать картинки и восстанавливать их неточно. С первого взгляда, решение этой задачи не приносит большой пользы, но, решив её, мы получили несколько интересных побочных эффектов.

Первым полезным применением нашей модели является поиск схожих изображений по сгенерированным кодам картинок.

Сперва закодируем наши изображения(не разкодируя обратно в картинки). Затем найдём близкие векторы-коды в нашей базе и покажем соответствующие им изображения как поисковую выдачу.

Импользуем локально чувствительное хэширование(LSH) для ускорения процесса поиска. Для простоты, возьмём <a href="http://scikit-learn.org/0.18/modules/generated/sklearn.neighbors.LSHForest.html#sklearn.neighbors.LSHForest"> реализацию из scikit-learn</a>

In [ ]:
# закодируем изображения
images = X_train
codes = autoencoder.encode(images, batch_size=10)

In [ ]:
# build hashes
lshf = LSHForest(n_estimators=50).fit(codes)

In [ ]:
# Функция нахождения схожих изображений
def get_similar(image, n_neighbors=5):
    assert image.ndim==3,"image must be [batch,height,width,3]"

    code = autoencoder.encode(image[None])
    
    (distances,),(idx,) = lshf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,images[idx]

In [ ]:
# Визуализация похожих изображений
def show_similar(image):
    
    distances,neighbors = get_similar(image,n_neighbors=11)
    
    plt.figure(figsize=[8,6])
    plt.subplot(3,4,1)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    plt.imshow(image)
    plt.title("Original image")
    
    for i in range(11):
        plt.subplot(3,4,i+2)
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off')

        plt.imshow(neighbors[i])
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:
# улыбки
show_similar(X_test[1])

In [ ]:
# Национальность
show_similar(X_test[499])

In [ ]:
# очки
show_similar(X_test[63])

## Преобразование картинок
Вторым, менее полезным, но не менее наглядным примером использования нашей модели будет нахождение промежуточных изображений при переходе от одной картинки к другой

In [ ]:
N_INTERMEDIATE = 8
for _ in range(5):
    image1,image2 = X_test[np.random.randint(0,len(X_test),size=2)]

    code1, code2 = autoencoder.encode(np.stack([image1,image2]))

    plt.figure(figsize=[10,4])
    plt.subplot(1,N_INTERMEDIATE+2,1)
    plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off')
    plt.imshow(image1)
    plt.title("original")
    for i,a in enumerate(np.linspace(0,1,endpoint=True,num=N_INTERMEDIATE)):

        output_code = code1*(1-a) + code2*(a)
        output_image = autoencoder.decode(output_code[None])[0]
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off')
        plt.subplot(1,N_INTERMEDIATE+2,i+2)
        plt.imshow(output_image)
        plt.title("a=%.2f"%a)
    plt.subplot(1,N_INTERMEDIATE+2,N_INTERMEDIATE+1)
    plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off')
    plt.imshow(image2)
    plt.title("target")
    plt.show()

## Преобразуем изображения на основе побочных данных картинок
Наша модель умеет восстанавливать изображение, по его закодированному вектору. Кодированный вектор несёт в себе много осмысленной информации. Мы можем манипулировать таким вектором, чтобы манипулировать хранимой в нём информацией.

Помимо лиц, наш датасет имеет набор атрибутов - значений, характеризующих дополнительную инфрмацию о картинке

Используем эту информацию для осмысленной манипуляцией над изображениями
<img src="http://www.samyzaf.com/ML/nlp/word2vec2.png">
    пример представления связей объектов в векторном пространстве


In [ ]:
# какие есть атрибуты
attr.columns

In [ ]:
# закодируем изображения
encoded = autoencoder.encode(X)

In [ ]:
attribute = 'Smiling' # аттрибут, который мы будем менять
# Попробуй 'Smiling', 'Strong Nose-Mouth Lines','Male', 'Black', 'Asian', 'Attractive Woman', 'Big Nose', Mustache'

mean_featured = (encoded * attr[attribute].as_matrix()[:encoded.shape[0],None].astype('float32')).mean(axis = 0)
mean_code = encoded.mean(axis = 0)

featured_direction = mean_featured - mean_code

In [ ]:
attr['Mustache'].astype('float32').idxmax()

In [ ]:
# выберем фото из датасета
def plot_morphing(factor, index):
    #factor = 2 # насколько сильно мы меняем картинку

    img = X[index]
    code = encoded[index]
    plt.subplot(1,3,1)
    plt.imshow(img) # выводим оригинальное изображение
    code_open = code + featured_direction*factor
    plt.subplot(1,3,2)
    plt.imshow(autoencoder.decode([code])[0])
    plt.subplot(1,3,3)
    plt.imshow(autoencoder.decode([code_open])[0]);
layout = widgets.Layout(width='100%', height='80px')
widgets.interact(plot_morphing, factor = widgets.FloatSlider(min=-10.0,max=10.,step= 0.1,layout=layout),
                 index = widgets.IntSlider(min=0,max=X.shape[0], step=1, layout=layout));

## загрузи свою картинку
загрузи её в папку с тетрадкой и укажи путь к ней в поле внизу 

In [ ]:
_left = _right = _bottom = _top = 0.5 # значения по умолчанию

In [ ]:
img = X[0]
def load_photo(path, left, bottom, right, top):
    #try:
    if True:
        pic = io.imread(path)
        global _left;global _right; global _bottom; global _top
        _left = left
        _right = right
        _bottom = bottom
        _top = top
        left = int(pic.shape[1] * left)
        top = int(pic.shape[0] * top)
        right = left + int((pic.shape[1] - left) * right)
        bottom = top + int((pic.shape[0] - top) * bottom)
        cropped = pic[top:bottom,left:right]
        cropped = transform.resize(cropped, img_shape, anti_aliasing=True)
        #pic = skimage.util.(pic, img_shape)

        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(pic)


        # Create a Rectangle patch
        rect = patches.Rectangle((left,top),right - left, bottom - top,linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

        plt.show()
        plt.imshow(cropped)
        global img;
        img = cropped
    #except:
    #    pass
widgets.interact(load_photo, path = "your_image.jpg",
                 left = widgets.FloatSlider(value = _left,min=0.,max=1.,step= 0.01,continuous_update=False,layout=layout),
                 right = widgets.FloatSlider(value = _right,min=0.01,max=1.,step= 0.01,continuous_update=False,layout=layout),
                 bottom = widgets.FloatSlider(value = _bottom,min=0,max=1.,step= 0.01,continuous_update=False,layout=layout),
                 top = widgets.FloatSlider(value = _top,min=0.01,max=1.,step= 0.01,continuous_update=False,layout=layout))

In [ ]:
visualize(img, autoencoder)

In [ ]:
<Попробуй сделать преобразования с лицом c твоей картинки>

In [ ]:
plt.imshow(img)

# Ура!

## Что делать дальше?
1. Можно посмотреть в эти же тетрадки дома и разобраться более детально. [Вот](https://github.com/tvorogme/digitalfest) репозиторий!
2. Прочитай [блог школы GoTo](https://habrahabr.ru/company/goto/blog/339050/), рассказывающий, с чего начинать изучать анализ данных и машинное обучение.
3. Когда ты наберёшься знаний и тебе захочется проверить свои силы, попробуй поучаствовать в соревнованиях на [kaggle](https://www.kaggle.com/competitions)

    ![](http://www.setwalls.ru/pic/201305/1680x1050/setwalls.ru-43884.jpg)

4. Когда ты научишься самостоятельно обучать нейронные сети, CPU для вычислений начнёт не хватать. Подумай о покупке GPU. Подойдёт любая CUDA-совместимая видеокарта, но чем мощнее - тем лучше
![](screens/zmubBCUZwBg.jpg)